In [28]:
import pandas as pd
import numpy as np
import os
import json
# import gui
from collections import Counter
import re

pd.set_option()

# Create a pandas dataframe of all cards

In [73]:
# Pull the data from the source json file
fp = "default-cards-20221230100442.json"
df_source = pd.read_json(fp, orient='records')

# Subset the source data into only fields we care about
lexicon = df_source[[
    'id',
    'name',
    'mana_cost',
    'cmc',
    'type_line',
    'oracle_text',
    'power',
    'toughness',
    'colors',
    'color_identity',
    'produced_mana',
    'keywords',
    'set',
    'rarity',
    'flavor_text',
    'edhrec_rank',
    'arena_id',
    'loyalty',
    'card_faces',
]]

df_deck = pd.DataFrame(columns=lexicon.columns)

### Create column for casting cost pips

In [79]:
def count_pips(s):
    """Applied to the mana_cost column in lexicon to return the number of pips in that color

    Args:
        s (string): mana cost separated by brackets

    Returns:
        pips (Counter): a Counter of the number of pips in each color W U B R G
    """
    
    s = s.replace("{", "").replace("}", "")

    colors = ["W", "U", "B", "R", "G"]
    W, U, B, R, G = 0, 0, 0, 0, 0
    pips = []
    pips = Counter(s)

    # Delete unecessary information
    to_delete = []
    for e in pips:
        if e not in colors:
            to_delete.append(e)
    for e in to_delete:
        del pips[e]


    return pips
# print(count_pips("WUBRG")["R"])

1


In [75]:
# A copy of the lexicon df is created so that we can avoid making changes to a copy of a sliced df
lexicon = lexicon.copy()
lexicon.loc[:, 'cast_cost_W'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['W'], na_action='ignore')
lexicon.loc[:, 'cast_cost_U'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['U'], na_action='ignore')
lexicon.loc[:, 'cast_cost_B'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['B'], na_action='ignore')
lexicon.loc[:, 'cast_cost_R'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['R'], na_action='ignore')
lexicon.loc[:, 'cast_cost_G'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['G'], na_action='ignore')


In [76]:
lexicon

,id,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,...,flavor_text,edhrec_rank,arena_id,loyalty,card_faces,cast_cost_W,cast_cost_U,cast_cost_B,cast_cost_R,cast_cost_G
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,[R],[R],...,"""A rift opened, and our arrows were abruptly s...",5708.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,[W],[W],...,"""We take only what we need to survive. Believe...",14503.0,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0
2,0000a54c-a511-4925-92dc-01b937f9afad,Spirit,,0.0,Token Creature — Spirit,Flying,1,1,[W],[W],...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,[U],[U],...,NaN,12608.0,66119.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0
4,00012bd8-ed68-4978-a22d-f450c8a6e048,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,[G],[G],...,NaN,19496.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78330,fffc0bfd-8e64-44bf-ae0a-5d2ee54c58df,Boomerang,{U}{U},2.0,Instant,Return target permanent to its owner's hand.,NaN,NaN,[U],[U],...,NaN,6083.0,NaN,NaN,NaN,0.0,2.0,0.0,0.0,0.0
78331,fffcbc4e-c6dc-4808-b262-f7c453e74dd8,Celestine Reef,,0.0,Plane — Luvion,Creatures without flying or islandwalk can't a...,NaN,NaN,[],[],...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
78332,fffce2f7-b619-4483-a75e-916343194641,Horned Troll,{2}{G},3.0,Creature — Troll,{G}: Regenerate Horned Troll.,2,2,[G],[G],...,Sword hilts jut from some trolls' bodies where...,21393.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
78333,fffe7b2b-22c3-4e6a-9b1b-c6d7b29b9f86,Exultant Skymarcher,{1}{W}{W},3.0,Creature — Vampire Soldier,Flying,2,3,[W],[W],...,"""We have come at last to this holiest of holy ...",17742.0,66631.0,NaN,NaN,2.0,0.0,0.0,0.0,0.0
